# AutoMorphoTrack Full Package Code\n\nThis notebook contains the full code for the AutoMorphoTrack package, including detection, morphology classification, motion analysis, colocalization, and example usage.

In [ ]:
# Detection module\nimport numpy as np\nfrom skimage.filters import threshold_isodata\nfrom skimage.measure import label, regionprops\nfrom skimage.morphology import remove_small_objects\n\ndef threshold_and_detect(image_stack, min_size=5):\n    thresh_stack = np.array([frame > threshold_isodata(frame) for frame in image_stack])\n    masks = [label(remove_small_objects(frame, min_size=min_size)) for frame in thresh_stack]\n    props_list = [regionprops(mask) for mask in masks]\n    return props_list

In [ ]:
# Morphology module\nimport pandas as pd\n\ndef classify_mitochondria_morphology(props_list):\n    morphology_per_frame = []\n    for i, props in enumerate(props_list):\n        elongated = 0\n        punctate = 0\n        for prop in props:\n            area = prop.area\n            eccentricity = prop.eccentricity\n            if area >= 0.2 and eccentricity >= 0.85:\n                elongated += 1\n            elif area >= 0.03 and eccentricity <= 0.7:\n                punctate += 1\n        morphology_per_frame.append({'Frame': i, 'Elongated': elongated, 'Punctate': punctate})\n    return pd.DataFrame(morphology_per_frame)

In [ ]:
# Motion module\nfrom scipy.spatial import distance\n\ndef calculate_motion(props_list):\n    tracks = [[prop.centroid for prop in props] for props in props_list]\n    displacements = []\n    velocities = []\n    for i in range(1, len(tracks)):\n        prev_centroids = tracks[i - 1]\n        curr_centroids = tracks[i]\n        frame_displacements = []\n        for curr in curr_centroids:\n            if not prev_centroids:\n                frame_displacements.append(0)\n                continue\n            dists = [distance.euclidean(curr, prev) for prev in prev_centroids]\n            frame_displacements.append(np.min(dists))\n        displacements.append(np.mean(frame_displacements) if frame_displacements else 0)\n        velocities.append(np.mean(frame_displacements) if frame_displacements else 0)\n    return pd.DataFrame({'Frame': range(1, len(displacements) + 1), 'Mean Displacement': displacements, 'Mean Velocity': velocities})

In [ ]:
# Colocalization module\nfrom scipy.stats import pearsonr\n\ndef calculate_colocalization_pearson(stack_mito, stack_lyso):\n    return [pearsonr(mito.flatten(), lyso.flatten())[0] for mito, lyso in zip(stack_mito, stack_lyso)]

In [ ]:
# Full stack processing utility\nimport tifffile\nimport os\n\ndef process_stack_all(path, treatment):\n    stack_name = os.path.basename(path).replace('.tif', '')\n    image_stack = tifffile.imread(path)\n    mito_stack = image_stack[:, 1, :, :]\n    lyso_stack = image_stack[:, 0, :, :]\n\n    props_list = threshold_and_detect(mito_stack)\n    df_morphology = classify_mitochondria_morphology(props_list)\n    df_morphology['Stack'] = stack_name\n    df_morphology['Treatment'] = treatment\n\n    df_motion = calculate_motion(props_list)\n    df_motion['Stack'] = stack_name\n    df_motion['Treatment'] = treatment\n\n    colocalization_scores = calculate_colocalization_pearson(mito_stack, lyso_stack)\n    \n    return df_morphology, df_motion, colocalization_scores

## Example Usage

In [ ]:
# Example analysis\n# df_morphology, df_motion, colocalization_scores = process_stack_all('example_file.tif', 'PFF+DMSO')\n\n# print(df_morphology)\n# print(df_motion)\n# print(colocalization_scores)